In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.linalg
from torch_geometric.nn import GCNConv
import numpy as np
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [3]:
# from torch_geometric.datasets import Planetoid
# name_data = 'Cora'
# dataset = Planetoid(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import Actor
# dataset = Actor(root= '/tmp/' + "Actor")

# from torch_geometric.datasets import WebKB
# name_data = 'Wisconsin'
# dataset = WebKB(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import Planetoid
# name_data = 'CiteSeer'
# dataset = Planetoid(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WebKB
# name_data = 'Cornell'
# dataset = WebKB(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WebKB
# name_data = 'Texas'
# dataset = WebKB(root= '/tmp/' + name_data, name = name_data)

from torch_geometric.datasets import WikipediaNetwork
name_data = 'Chameleon'
dataset = WikipediaNetwork(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WikipediaNetwork
# name_data = 'Squirrel'
# dataset = WikipediaNetwork(root= '/tmp/' + name_data, name = name_data)

In [4]:
data = dataset[0]

edges = data.edge_index

#dimension: num_nodes * num_features
features = data.x

num_nodes = features.shape[0]
num_features = features.shape[1]
#dimension: num_nodes * num_nodes
adj = torch.eye((num_nodes))
degree = torch.zeros((num_nodes, num_nodes))
for i in tqdm(range(edges.shape[1])):
    first = edges[0][i]
    second = edges[1][i]
    adj[first][second] = 1
    adj[second][first] = 1
for i in tqdm(range(num_nodes)):
    degree[i][i] = sum(adj[i][:])

100%|██████████| 2277/2277 [00:34<00:00, 66.61it/s]


In [5]:
sample_num = int(num_nodes*0.6)
sample_lst = np.random.choice(num_nodes,sample_num, replace = False)
train_mask = [False] * num_nodes
test_mask = [True] * num_nodes
for i in sample_lst:
  train_mask[i] = True
  test_mask[i] = False
train_mask = torch.tensor(train_mask)
test_mask = torch.tensor(test_mask)
degree = degree.to(device)
adj = adj.to(device)
data = data.to(device)
features = features.to(device)

In [6]:
print(num_nodes)
print(num_features)
print(dataset.num_node_features)

2277
2325
2325


In [7]:
class FBLayer(nn.Module):
    def __init__(self, dim_in, dim_out):
      super().__init__()
      self.high = nn.Linear(dim_in, dim_out, bias = False)
      gain = nn.init.calculate_gain("relu")
      nn.init.xavier_normal_(self.high.weight, gain)
      self.activation = nn.ReLU()
        
    def forward(self, input, lap, d_inv):
      Lhp = torch.mm(torch.mm(d_inv, lap), d_inv)
      Hh = torch.mm(Lhp, self.activation(self.high(input)))
      return Hh

In [8]:
class GCN(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.conv1 = GCNConv(dataset.num_node_features, 16)
      self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
      x, edge_index = data.x, data.edge_index
      x = self.conv1(x, edge_index)
      x = F.relu(x)
      x = F.dropout(x, training=self.training)
      x = self.conv2(x, edge_index)

      return F.log_softmax(x, dim=1)

In [9]:
class FBGCN(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.fb1 = FBLayer(dataset.num_node_features, 16)
      self.fb2 = FBLayer(16 ,dataset.num_classes)
      self.conv1 = GCNConv(dataset.num_node_features, 16)
      self.conv2 = GCNConv(16, dataset.num_classes)
      self.aL_1 = torch.nn.Parameter(torch.rand(1))
      self.aH_1 = torch.nn.Parameter(torch.rand(1))
      self.aL_2 = torch.nn.Parameter(torch.rand(1))
      self.aH_2 = torch.nn.Parameter(torch.rand(1))

    def forward(self, data, lap, d_inv):
      input, edge_index = data.x, data.edge_index
      x_1 = self.fb1(input, lap, d_inv) #high pass
      y_1 = self.conv1(input, edge_index) #low pass layer
      z_1 = self.aH_1 * x_1 + self.aL_1 * y_1
      z_1 = F.relu(z_1)
      z_1 = F.dropout(z_1, training=self.training)
      x_2 = self.fb2(z_1, lap, d_inv)
      y_2 = self.conv2(z_1, edge_index)
      z_2 = self.aH_2 * x_2 + self.aL_2 * y_2

      return F.log_softmax(z_2, dim=1)

In [10]:
# acc_list_2 = []
# lap = degree - adj
# inter = scipy.linalg.fractional_matrix_power(degree.cpu(), (-1/2))
# d_inv = torch.from_numpy(inter).to(device)
# for i in tqdm(range(5)):
#     model_2 = GCN().to(device)
#     model_2.train()
#     optimizer = torch.optim.Adam(model_2.parameters(), lr=0.05 ,weight_decay=5e-5)
#     for epoch in range(100):
#         optimizer.zero_grad()
#         out = model_2(data)
#         loss = F.nll_loss(out[train_mask], data.y[train_mask]) 
#         loss.backward()
#         optimizer.step()
#     model_2.eval()
#     _, pred = model_2(data).max(dim=1)
#     correct = int(pred[test_mask].eq(data.y[test_mask]).sum().item())
#     acc = correct / int(test_mask.sum())
#     acc_list_2.append(acc)
#     del model_2
#     del correct
#     torch.cuda.empty_cache()
# del d_inv, inter, lap
# print('FBGCN Accuracy: {:.4f}'.format(np.mean(acc_list_2)))

In [11]:
acc_list = []
lap = degree - adj
inter = scipy.linalg.fractional_matrix_power(degree.cpu(), (-1/2))
d_inv = torch.from_numpy(inter).to(device)
for i in range(1):
    model = FBGCN().to(device)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.05 ,weight_decay=5e-5)
    for epoch in tqdm(range(100)):
        optimizer.zero_grad()
        out = model(data, lap, d_inv)
        loss = F.nll_loss(out[train_mask], data.y[train_mask]) 
        loss.backward()
        optimizer.step()
    model.eval()
    _, pred = model(data, lap, d_inv).max(dim=1)
    correct = int(pred[test_mask].eq(data.y[test_mask]).sum().item())
    acc = correct / int(test_mask.sum())
    acc_list.append(acc)
    del correct
    torch.cuda.empty_cache()
del d_inv, inter, lap
del model
print('FBGCN Accuracy: {:.4f}'.format(np.mean(acc_list)))

100%|██████████| 100/100 [00:15<00:00,  6.49it/s]
FBGCN Accuracy: 0.4786


In [12]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)

NameError: name 'model' is not defined

In [26]:
# for layer in model.children():
#    if hasattr(layer, 'reset_parameters'):
#        layer.reset_parameters()